The aim of this lab is to develop a Language Model based on n-gram.

We need to load the data from `data/iwslt-en-de-preprocessed.tar.gz`. The dataset we are going to use comes from German-English translation task in [IWSLT campaign](http://phontron.com/data/iwslt-en-de-preprocessed.tar.gz).

In [93]:
import tarfile
import re

from math import log
from collections import Counter, defaultdict
from itertools import chain

import pandas

## First step: preparing data

In [67]:
def preprocess_data(filename):
  # The file is around 10MB. Don't care about buffer.
  with tarfile.open(name=filename, mode='r') as tar:
    EXTRACTION = ["train", "valid", "test"]
    for t in EXTRACTION:
        with open(f"data/ngram/{t}.txt", 'wb') as out:
          f = tar.extractfile(f"en-de/{t}.en-de.en")
          out.write(f.read())

In [68]:
preprocess_data("data/iwslt-en-de-preprocessed.tar.gz")

It's good to check how the data looks like.

In [69]:
with open("data/ngram/train.txt") as f:
    print(f.readline(1000))

With vibrant video clips captured by submarines, David Gallo takes us to some of Earth's darkest, most violent, toxic and beautiful habitats, the valleys and volcanic ridges of the oceans' depths, where life is bizarre, resilient and shockingly abundant.



## Second step: learning parameters

Remember our sequence is defined as $E = e_1, e_2, \ldots, e_t = e_1^T$ where $|T|$ is the length of a sequence. 

Firstly, we can have a word-by-word computation of probabilties as 
$$P(E)=P\left(|E|=T, e_{1}^{T}\right)$$
where the length of $E$ is $|E|=T$.

This is hard because the length of a sentence is not predetermined and there are a lage number of possible combinations of words.

While the word-by-word model would be simpler as
$$P(E)=\prod_{t=1}^{T+1} P\left(e_{t} | e_{1}^{t-1}\right)$$
where $e_{T+1}=\langle / s\rangle$ is an end symbol.

### How to count?

The vanilla approach for counting would be to count from the start point of each sentences with a particular
context. If we limit ourself in a sentence level, we can apply maximum likelihood estimation (MLE)

$$P_{\mathrm{ML}}\left(e_{t} | e_{1}^{t-1}\right)=\frac{c_{\text { prefix }}\left(e_{1}^{t}\right)}{c_{\text { prefix }}\left(e_{1}^{t-1}\right)}$$

Once we limit the model to look at a fixed window of previous words,
we would have got a better context

$$\theta_{e_{t-n+1}^{t}}=P_{\mathrm{ML}}\left(e_{t} | e_{t-n+1}^{t-1}\right)=\frac{c\left(e_{t-n+1}^{t}\right)}{c\left(e_{t-n+1}^{t-1}\right)}$$

where c means counting.


In [70]:
# How to clean data also depends on what you want to achieve.
# Let's keep the data messy.

START_SYMBOL='<s>'
STOP_SYMBOL='</s>'

def tokenize(l):
  for b in l.rstrip().split(' '):
    for c in re.split(r'(\W+)', b):
      if c != '':
        yield(c)

def get_vocabs(*files):
  vocab_set = set()

  for filename in files:
    with open(filename, 'r') as f:
      for l in f:
        vocab_set.update(tokenize(l))

  if "" in vocab_set:
    vocab_set.remove("")
    
  return list(vocab_set | set([START_SYMBOL, STOP_SYMBOL]))

In [71]:
list(tokenize("a b c."))

['a', 'b', 'c', '.']

In [72]:
VOCABS = get_vocabs("data/ngram/train.txt", "data/ngram/valid.txt", "data/ngram/test.txt")
VOCABS

['Ronnefeldt',
 'Scrapper',
 'polymers',
 'Stand',
 'Paulista',
 'buildup',
 'tagline',
 'faggot',
 'valence',
 'Ubirajara',
 'Aveling',
 'Deal',
 'raced',
 'bunny',
 'latticework',
 'painters',
 'resin',
 'courses',
 'wretches',
 'encompassing',
 'Wiltshire',
 'distinctive',
 'MPAs',
 'pioneered',
 'movements',
 'antiquated',
 'Capitol',
 'mating',
 'Of',
 'Knows',
 'aaah',
 'disposability',
 'cathedrals',
 'wheatgrass',
 'deceptions',
 'typesetted',
 'infallibly',
 'versus',
 'engaged',
 'Odd',
 'Points',
 'corrupts',
 'phytoplankton',
 'demanding',
 'heartbreaking',
 'km²',
 'speaker',
 'head',
 'mingle',
 'Erez',
 'censor',
 'Arby',
 'USFDA',
 'facilitated',
 'fantastic',
 'Matchstick',
 'scorpion',
 'extracellular',
 'bitching',
 'Anywhere',
 'Olympicopolis',
 'scourge',
 'views',
 '630',
 'astrocytes',
 'rollout',
 'madcap',
 'paleoanthropology',
 'Jeeves',
 'inhere',
 'coffers',
 'advantages',
 'conquesting',
 'explained',
 'Director',
 'Him',
 'fjord',
 'Sue',
 'inventing',
 'd

We needs to count now. Let's use standard paddings.
In the training data file, each line is a sentence.

<small>Question: How many parameters we have for a particular n-gram models? (Try come up with your own symbols with reasoning)</small>

In [73]:
def pad_sequence(
    sequence,
    n
):
  LEFT_PAD_SYMBOL = '<s>'
  RIGHT_PAD_SYMBOL = '</s>'
  return chain((LEFT_PAD_SYMBOL,) * (n - 1), iter(sequence), (RIGHT_PAD_SYMBOL,) * (n - 1))

def ngram(sequence, n):
  sequence = pad_sequence(sequence, n)

  history = []
  while n > 1:
    try:
      next_item = next(sequence)
    except StopIteration:
      # no more data, terminate the generator
      return
    history.append(next_item)
    n -= 1
  
  for item in sequence:
    history.append(item)
    yield tuple(history)
    del history[0]

In [74]:
list(ngram(tokenize("It's a beautiful day."), n=2)) # try change n

[('<s>', 'It'),
 ('It', "'"),
 ("'", 's'),
 ('s', 'a'),
 ('a', 'beautiful'),
 ('beautiful', 'day'),
 ('day', '.'),
 ('.', '</s>')]

<small>Answer to size of parameters: Say we have a corpus which has many sentences,
their lengths are $T_1, T_2, \ldots, T_n$. We have $n-1$ paddings.

Firstly, we calculate the number of tokens.

For uni-gram, $\sum_{i=1}^{n}T_i$ is obvious. Say the number of unique tokens is $Q_1$.

For bigram, we have $\sum_{i=1}^{n}T_i+1$ tokens. Say the number of unique tokens is $Q_2$.

For trigram, we have $\sum_{i=1}^{n}T_i+2$ tokens. Say the number of unique tokens is $Q_3$.

For ngrams, we have $\sum_{i=1}^{n}T_i+n-1$ tokens. Say the number of unique tokens is $Q_n$.

But model parameters are different.
Uni-gram is special, it has the number of unique tokens from uni-gram $Q_1$.
Bigram will have $Q_1 + Q_2$, ngram will have $Q_n+Q_{n-1}$.
</small>

### Build a Probability

Once we have the counting and we have symbols, we could define probabilities.

In [75]:
def train(filename):
  unigram_cnts = Counter()
  bigram_cnts = Counter()
  trigram_cnts = Counter()

  with open(filename, 'r') as f:
    for l in f:
      unigram_cnts.update(ngram(tokenize(l), 1))
      bigram_cnts.update(ngram(tokenize(l), 2))
      trigram_cnts.update(ngram(tokenize(l), 3))

  return unigram_cnts, bigram_cnts, trigram_cnts

In [76]:
u, b, t = train("data/ngram/train.txt")

In [77]:
# let's feel the size of parameters
print(len(VOCABS), len(u), len(b), len(t))

43314 42855 468743 1153478


## Third step: now we can test it 

In [120]:
def total_counts(cnts):
  return float(sum([cnt for _, cnt in cnts.items()]))

def freq(cnts):
  total = total_counts(cnts)
  result = defaultdict(float)
  for k, cnt in cnts.items():
    result[k] = cnt / total
  return result


def test_uni_normal(filename, unigram_cnts):
  unigram_freq = freq(unigram_cnts)

  with open(filename, 'r') as f:
    for l in f:
      p = 1.0
      for t in ngram(tokenize(l), 1):
        p *= unigram_freq[t]
      print(p)
      

In [121]:
# Now we have unigram probability (what would happen if float underflows)
test_uni_normal('data/ngram/test.txt', u)

6.734228559893649e-88
1.162249812275207e-81
0.0
0.0
2.1297919467535287e-175
0.0
8.571040468736674e-70
0.0
1.5318831840359566e-88
0.0
0.0
0.0
4.914419669280988e-61
0.0
2.5762438815298032e-17
0.0
2.702168688328846e-56
5.5320965327361885e-80
3.787047465679271e-58
0.0
0.0
3.5524466244807593e-71
1.902801512691112e-67
5.176925614481082e-139
2.3973531497037667e-26
6.8393796175362375e-25
4.636217180439505e-72
1.7825770518186525e-30
1.5542845663583395e-30
6.15795458205408e-139
5.90669392512205e-55
3.125813356445122e-42
0.0
0.0
2.668834465798045e-07
4.482286073647531e-26
3.8264505509886376e-12
0.0
8.912991336090585e-18
1.0654354720382116e-33
1.4498381344464403e-18
2.2063646360396205e-33
0.0
7.595514317943907e-11
1.5332852527330406e-48
1.3892563927073018e-24
1.6806119343181376e-32
2.6695411920890454e-69
1.1643973229365906e-17
6.34179488262617e-83
3.4678644598377106e-58
1.1453887968190045e-25
0.0
1.359767382891048e-19
2.437429150921278e-60
0.0
1.937922633768812e-37
2.4319495413441963e-59
6.6805883

0.0
8.951861882764908e-86
2.4749487026315302e-45
2.5380710430268297e-99
1.2411217960340242e-48
4.380457876284546e-33
0.0
2.809536638579522e-85
6.194425400042211e-83
6.532973685035095e-47
1.4687051325884774e-52
1.0910383971857538e-18
0.0
0.0
0.0
6.260724973244208e-25
0.0
4.960967148067049e-22
6.956111452844635e-42
0.0
3.393399037521703e-42
3.324516087570725e-58
1.6748587146787244e-30
0.0
4.8910511244291025e-68
7.469899721012071e-82
3.4733923166672443e-34
7.670869143627167e-13
7.454539979329786e-45
8.905512678088428e-71
5.913416249358068e-97
6.463388620097609e-70
4.011392010997204e-32
7.95218215283166e-35
1.2692500405977227e-33
7.460124903547982e-44
0.0
0.0
7.765118141807203e-38
0.0
0.0
5.943391405358801e-66
2.0809079453757263e-25
2.1084928774777726e-36
0.0
1.527107519426071e-119
3.530174587084837e-56
0.0
2.021760313208573e-40
2.1705253671971715e-35
2.5063503014812547e-103
6.351916805495503e-24
1.1087652742551367e-13
0.0
3.120838043309799e-25
2.1299519119257257e-60
1.957787399067172e-132

In [122]:
def test_uni(filename, unigram_cnts):
  unigram_freq = freq(unigram_cnts)

  with open(filename, 'r') as f:
    for l in f:
      p = 0.0
      for t in ngram(tokenize(l), 1):
        if unigram_freq[t] == 0.0:
          p += 0.0
        else:
          p += log(unigram_freq[t])
      print(p)

test_uni('data/ngram/test.txt', u)

-200.72028492213494
-186.35903491244753
-513.6882744664451
-230.1529954628625
-402.196366976588
-127.28708021703204
-159.03256737608888
-355.0446706198215
-202.2009903656941
-237.53781632440186
-353.9276465759647
-257.3413913763101
-138.8655169994003
-381.0852973218865
-38.1976141026311
-61.244971461323026
-127.95071053916355
-182.4962405759934
-132.21834871096635
-234.77222665423236
-159.56588304766555
-162.21590504647767
-153.6298739500437
-318.41511655681614
-58.99284714337268
-55.641930296520705
-164.2522279246769
-68.49949269062257
-68.63653743600537
-318.24158325213523
-124.86609384331167
-95.56887938236454
-168.1088672895708
-234.97995691055576
-15.136453803516627
-58.36707921727273
-26.289083491522565
-112.69414569166575
-39.25902176079783
-75.92192445928387
-41.07507975510783
-75.19396186877589
-87.86151068435096
-23.30087817121456
-110.09667180629214
-54.93327359713693
-73.16356500209805
-157.89646479711175
-38.991742947104676
-189.267400885608
-132.30639641859187
-57.4288831

-337.8176473664897
-145.7974738488788
-136.36975009392836
-130.4534033135101
-93.5843218354434
-102.2485470446786
-112.93651869467378
-146.64579356591705
-123.38486255132293
-296.0552247309255
-159.22910094532205
-31.530663303254286
-75.35754733409361
-187.4892592248463
-137.43787016897252
-51.7571252014745
-133.2453854598731
-152.80413424345136
-140.61464201709336
-55.715363093651455
-129.32202671721467
-86.26329077353232
-198.1090920787578
-62.810561669707845
-284.85844683353974
-46.517853813668765
-174.5984202646077
-59.839331539599826
-62.96212704347248
-165.27826211076996
-49.43538509410035
-293.11597369853706
-211.66393628965005
-68.58960772395731
-42.335664678415995
-358.982067691447
-88.77386482902972
-248.47712437268783
-245.1202921957513
-103.44012758439939
-124.70826475180665
-246.26312892842822
-97.2845035017556
-141.1516614557621
-107.46704608889223
-130.1898675763164
-146.8724793083162
-117.77992685747694
-143.6778099572908
-69.88068492928726
-285.27252300178554
-77.05413

Now we try interpolation:

$$ P\left(e_{t} | e_{t-1}\right)=(1-\alpha) P_{\mathrm{ML}}\left(e_{t} | e_{t-1}\right)+\alpha P_{\mathrm{ML}}\left(e_{t}\right) $$

In [123]:
def test_interpolation(filename, unigram_cnts, bigram_cnts, alpha=0.1):
  unigram_freq = freq(unigram_cnts)
  bigram_freq = freq(bigram_cnts)

  with open(filename, 'r') as f:
    for l in f:
      p = 0.0
      for t in ngram(tokenize(l), 2):
        term = (1-alpha) * bigram_freq[t] + alpha * unigram_freq[t[-1]]
        if term != 0.0:
          p += log(term)
      print(p)

In [124]:
test_interpolation('data/ngram/test.txt', u, b)

-230.2567084780431
-265.22547635488166
-445.80568243326894
-251.77567092709808
-501.3488478846871
-139.95476254184433
-215.36236475322414
-340.56377621618907
-244.25188457565469
-257.7580219740459
-427.7644613706315
-289.8923346006746
-161.2738156046434
-444.0973167945744
-68.65977380104934
-65.99015908543473
-174.55632613101506
-226.28521968383302
-115.01464695421024
-278.4910897136805
-189.9251971795532
-249.44136255285122
-237.1248725128442
-419.234358768323
-63.24475417510948
-42.32133529303025
-247.39683046986875
-80.96953273919935
-121.13768390934288
-452.24788224188245
-146.05901309931113
-127.87329961845032
-225.29123389762782
-287.65769902499267
-26.918062782389608
-90.0602448429696
-45.50949062343256
-120.95400501167548
-48.61491772796352
-128.62186326725185
-58.66641441721273
-101.64494776807817
-75.80078133050952
-44.99213503702525
-134.97964040684917
-85.40382623003319
-114.18166166719527
-220.3436106928554
-55.75472215380867
-201.22238084143586
-171.3901694346133
-94.1135

-149.38742829775174
-359.3312761910602
-201.19598761592476
-360.31821884356435
-49.8835857176375
-239.4301728798564
-115.98901151113344
-131.96861867434586
-51.74988487317718
-102.10315934804741
-105.47028343824127
-132.87693286534784
-666.8141935011704
-100.90280932771662
-258.37628547677497
-170.3010472484961
-202.22871430688082
-139.64676258571086
-101.16975282716605
-379.4260641208893
-120.98458873853032
-195.31320175089877
-74.30860648062529
-222.80155698088458
-94.10620123464047
-576.4624405357789
-81.85140409507167
-152.0906786141644
-158.35191647146985
-75.70146390880035
-265.32242953409207
-310.7083966945609
-92.95384171888446
-108.26083878505906
-141.57577542588575
-398.69427572294114
-259.4300787171567
-165.20862736916087
-163.5024953776308
-65.04453138222561
-493.83927073236765
-104.64296838471174
-323.8358746603511
-71.84010574850868
-630.3755907470493
-189.9743928504122
-109.86940827050553
-156.26911478769296
-69.63959050323473
-139.8848591762503
-137.31875568448078
-88.4

We can do it recursively:

$$ P\left(e_{t} | e_{t-m+1}^{t-1}\right)=\left(1-\alpha_{m}\right) P_{\mathrm{ML}}\left(e_{t} | e_{t-m+1}^{t-1}\right)+\alpha_{m} P\left(e_{t} | e_{t-m+2}^{t-1}\right) $$.

In [142]:
def test_interpolation_more_context(filename, cnts, alpha=0.1):
  """cnts is a tuple of counter and n"""
  cnts = sorted(cnts, key=lambda x: x[1], reverse=True)
  freqs = [(freq(cnt), n) for cnt, n in cnts]

  large_freq, max_n = freqs[0]
  with open(filename, 'r') as f:
    for l in f:
      p = 0.0
      
      term = 0.0
      for t in ngram(tokenize(l), max_n):
        term = 0.0
        for f, n in freqs[1:]:
          term = f[t[-n:]] + term * alpha
        term = (1-alpha) * large_freq[t] + term * alpha
        
        if term != 0.0:
          p += log(term)
      print(p)

test_interpolation_more_context('data/ngram/test.txt', [(u, 1), (b, 2), (t, 3)], alpha=0.1)

-264.4337224918889
-259.03894416734653
-674.3205581863272
-306.96583012421644
-544.8579232437644
-175.7612598941206
-218.62444715680658
-471.7640754378334
-278.3745898408833
-323.54692115064586
-476.0626421451729
-346.5009052184727
-193.05513989527248
-523.1435166479318
-63.077934317261246
-92.87119035850527
-182.05264797333052
-255.220864552543
-180.1797453162642
-321.91752523734385
-234.48653052105044
-227.68675099987652
-213.32333007400527
-428.46606932456393
-86.92862657539483
-83.06722733289077
-226.19232820155727
-100.33690575646916
-104.23956263782458
-439.92068298979706
-176.8524593189057
-136.73903875422047
-232.26791293991786
-315.60226933486024
-30.39050905951636
-84.08103119307204
-43.74263776221893
-156.63448950131254
-61.550911954813074
-114.49357621219872
-65.7874184209417
-111.76581940520177
-130.83588162205947
-41.762848720493544
-155.63460511035348
-83.58483117163665
-110.5817442802247
-224.57894023916134
-63.432692730889755
-264.08201061020577
-184.79202061303286
-83

-314.4771075790913
-357.77284940409277
-146.05256003178064
-117.7975424213846
-655.814928418347
-153.10656423078822
-103.10680199326113
-386.36780672016505
-163.47043586378618
-333.7720255045364
-174.02737408817214
-194.87308548769118
-393.7837299317354
-173.646725414763
-70.9814795563197
-205.88133708607788
-112.33444596560362
-205.37827222754007
-95.08801968669708
-237.81039557699083
-201.46594701589183
-652.0794592446357
-92.52972921831643
-188.98723166833082
-393.85075734678526
-320.7460542671871
-87.62960965627413
-90.78113834709012
-337.946006563845
-446.35628452713644
-241.1751032800059
-126.55461998163875
-102.62485682422687
-195.8879461214722
-196.24449289123297
-126.3610898847851
-128.61231647564952
-664.2155812345004
-142.3917716304791
-604.2134442871775
-676.7126457888411
-351.9956202432242
-93.50503361329687
-379.24187264595037
-356.9454174406287
-231.99797363588794
-196.4224158810902
-75.87385886805224
-67.3846564118585
-177.7360236693582
-380.96450916443297
-119.08892177

-194.83487319766624
-400.4138279971547
-199.50491265197843
-195.14036236233989
-301.57349694430343
-270.1239579549757
-526.9477512073933
-301.2980816182007
-274.3526404367341
-117.49464179672424
-144.63164317197777
-174.83827978575977
-353.11517817984685
-388.7038007334139
-210.87684978765577
-337.5125735221188
-308.4133786327062
-76.54242828174397
-315.09533032341454
-129.2219351913194
-170.7546374375844
-184.17605688818992
-80.98381879814815
-136.9870241357662
-73.63942260245048
-120.41818092866171
-146.9619670734391
-69.06990709493009
-56.636350373846646
-272.10400210446227
-59.03899602222737
-447.76200272855823
-258.46335748125455
-389.73638346983773
-190.11087447465422
-311.613508405596
-92.90265870855009
-214.5728617800721
-168.43264117034875
-201.39761202741104
-120.68785599794025
-162.98743786863884
-48.8393444152888
-98.5543163151267
-361.0113142273619
-256.0815341121936
-378.6051273309155
-476.08123801636987
-712.6865902600015
-164.1395693615571
-206.28334070159005
-391.79028